# Problem 1


In [ ]:
import uaibot as ub
import numpy as np
from uaibot.utils import Utils
from uaibot.simobjects import Ball, Frame


kuka = ub.robot.Robot.create_kuka_kr5()
target_pos = Ball(radius=0.05)
target_ori = Frame()
sim = ub.Simulation.create_sim_grid([kuka, target_pos, target_ori])
target_htms = [Utils.htm_rand(trn_min=[-0.7,-0.7, 0], trn_max = [0.7,0.7,0.7]) for _ in range(50)]

q_max = kuka.joint_limit[:, 1]
q_min = kuka.joint_limit[:, 0]

dt = 0.01
K = 5*np.eye(6)

j = 0
old_r = np.array([np.inf] * 6).reshape(-1, 1)
max_iter = int(5 / dt)

fails = []

for n, H_target in enumerate(target_htms):
    print(f"{n}-th target")
    r = np.inf
    i = 0
    target_pos.add_ani_frame(time=j * dt, htm=H_target)
    target_ori.add_ani_frame(time=j * dt, htm=H_target)
    kuka.add_ani_frame(time=j * dt, q=kuka.q0)
    k = 0
    while np.linalg.norm(r) > 1e-2:
        if i > max_iter:
            print("Max iterations reached")
            fails.append(n)
            break
        q = kuka.q.copy()
        r, Jr = kuka.task_function(H_target)
        q_dot = -Utils.dp_inv(Jr) @ K @ r

        if (np.linalg.norm(old_r - r) <= 1e-3) and (np.linalg.norm(r) > 1e-1):
            k += 1
        else:
            k = 0 

        if k > 100:
            print("No improvement for 100 iterations")
            fails.append(n)
            break
        q_ = q + q_dot * dt

        violation_idxs = np.nonzero((q_ > q_max) | (q_ < q_min))[0]
        q_dot[violation_idxs] = 0

        q += q_dot * dt
        i += 1
        j += 1
        old_r = r

        kuka.add_ani_frame(time=j * dt, q=q)

# sim.run()
print(f"Strategy worked on {(len(target_htms) - len(fails))}/{len(target_htms)} targets ({(len(target_htms) - len(fails)) / len(target_htms)}%)")

# Problem 2

In [ ]:
import sys
import uaibot as ub
import numpy as np
from uaibot.utils import Utils
from uaibot.simobjects import Ball, Frame, PointCloud
from scipy.linalg import expm

def R_theta(theta, axis, type_='htm'):
    """Returns rotation matrix around axis by angle theta"""
    Q = np.eye(3) + np.sin(theta) * Utils.S(axis) + (1 - np.cos(theta)) * (Utils.S(axis) @ Utils.S(axis))
    if type_ == 'htm':
        H = np.eye(4)
        H[:3, :3] = Q
        H[:3, 3] = np.zeros(3)
        return np.array(H)
    else:
        return np.array(Q)

def pR2htm(p, R):
    p_ = np.array(p).ravel()
    if p_.shape[0] == 4:
        p_ = p_[:3]
    R_ = np.array(R)
    H = np.eye(4)
    H[:3, :3] = R_
    H[:3, 3] = p_[:3]
    return H

def progress_bar(i, imax):
    """Prints a progress bar in the terminal.

    Parameters
    ----------
    i : int
        Current iteration.
    imax : int
        Maximum number of iterations.
    """
    sys.stdout.write("\r")
    sys.stdout.write(
        "[%-20s] %d%%" % ("=" * round(20 * i / (imax - 1)), round(100 * i / (imax - 1)))
    )
    sys.stdout.flush()

def D_hat(V, W):
    """The Element-Element (EE-)distance function. This uses the algorithm in
    section 4.1 of the paper, and is an evaluation of ||log(V^-1 W)||_F.

    Parameters
    ----------
    V : np.array
        Homogeneous transformation matrix V (SE(3) element).
    W : np.array
        Homogeneous transformation matrix W (SE(3) element).
    
    Returns
    -------
    distance : float
        The EE-distance between V and W.
    """
    # Compute Z = V^-1 W
    Z = np.linalg.inv(V) @ W
    # Extract the rotation matrix
    Q = Z[:3, :3]
    # Extract the translation vector
    t = Z[:3, 3]
    # Compute the auxiliary variables
    u = 0.5 * (np.trace(Q) - 1) # This is cos(theta)
    v = 1/(2*np.sqrt(2)) * np.linalg.norm(Q - np.linalg.inv(Q), ord='fro') # This is sin(theta)
    # Compute theta with atan2
    theta = np.arctan2(v, u)
    # Recompute cos and theta to ensure correct values
    u = np.cos(theta)
    v = np.sin(theta)
    # Compute alpha
    alpha = (2 - 2*u - theta**2) / (4*(1 - u)**2)
    # Compute matrix M
    M = np.eye(3) * (1 - 2*alpha) + (Q + np.linalg.inv(Q)) * alpha
    # Compute the EE-distance
    distance = np.sqrt(2 * theta**2 + (t.T @ M @ t))
    return distance

def D(V, curve_parametrization):
    """The Element-Curve (EC-)distance function. This uses the brute force
    approach to compute the minimum distance between V and the curve
    parametrization hd(s). (See Definition 3.4 in paper). This function
    also returns the index of the curve that is closest to V for simplicity.

    Parameters
    ----------
    V : np.array
        Homogeneous transformation matrix V (SE(3) element).
    curve_parametrization : np.array
        Array with the precomputed curve. The shape is (n_points, 4, 4).
    
    Returns
    -------
    min_distance : float
        The minimum distance between V and the curve.
    min_index : int
        The index of the curve that is closest to V.
    """
    min_distance = np.inf
    min_index = -1
    for i, Hd_s in enumerate(curve_parametrization):
        distance = D_hat(V, Hd_s)
        if distance < min_distance:
            min_distance = distance
            min_index = i
    return min_distance, min_index

def S(xi):
    """S map in paper (See Definition 2.2). In this case we assume the basis
    as in section 4.3, since it will render the classical twist vector. This
    maps a vector xi into the Lie algebra se(3).

    Parameters
    ----------
    xi : np.array
        Twist vector xi.
    
    Returns
    -------
    lie_algebra : np.array
        The corresponding Lie algebra element.
    """
    xi = np.array(xi).ravel()
    lie_algebra = np.zeros((4, 4)) # Initialize the Lie algebra element
    # The last three elements of xi are assigned to the upper right 3x3 block
    lie_algebra[0, 1] = -xi[5]
    lie_algebra[0, 2] = xi[4]
    lie_algebra[1, 2] = -xi[3]
    # Since the orientation-related portion of the lie algebra is skew-symmetric,
    # we can compute it as (since the position-related portion is still zero):
    lie_algebra = lie_algebra - lie_algebra.T

    # The first three elements of xi are assigned to the last column
    lie_algebra[:3, 3] = xi[:3]
    return lie_algebra

def S_inv(lie_algebra):
    """The inverse of the S map. This maps a Lie algebra element to the twist
    vector. Here we also assume the basis as in section 4.3.

    Parameters
    ----------
    lie_algebra : np.array
        The Lie algebra element.
    
    Returns
    -------
    xi : np.array
        The corresponding twist vector xi.
    """
    xi = np.zeros(6) # Initialize the twist vector
    # The last three elements of xi are assigned from the upper right 3x3 block
    xi[3] = -lie_algebra[1, 2]
    xi[4] = lie_algebra[0, 2]
    xi[5] = -lie_algebra[0, 1]
    # The first three elements of xi are assigned from the last column
    xi[:3] = lie_algebra[:3, 3]
    return (xi).reshape(-1, 1)

def L(f, V, W, arg='V', epsilon=1e-3):
    """L operator (see Definition 2.6). This is a 'derivative' in the Lie group.
    Note that this is a portion of the Lie derivative.

    Parameters
    ----------
    f : function
        Differentiable function that receives two arguments, V and W. This
        function maps SE(3) x SE(3) -> R.
    V : np.array
        Homogeneous transformation matrix V (SE(3) element).
    W : np.array
        Homogeneous transformation matrix W (SE(3) element).
    arg : str or int, optional
        The argument to compute the derivative. The default is 'V'. The options
        are 'V' (0) or 'W' (1). If 'V' is selected, the derivative is computed
        with respect to V, and if 'W' is selected, the derivative is computed
        with respect to W.
    epsilon : float, optional
        Small value to compute the derivative. The default is 1e-3.
    
    Returns
    -------
    L_ : np.array
        The resulting L operator. This is a row vector of shape (1, 6).

    Raises
    ------
    ValueError
        If an invalid argument 'arg' is passed.
    """
    L_ = np.zeros((1, 6)) # Initialize the resulting L
    I = np.eye(6) # Initialize the identity matrix
    if arg.lower() == 'v' or arg == 0:
        curr_f = f(V, W)
        for j in range(6):
            zeta = I[:, j]
            next_f_j = f(expm(S(zeta) * epsilon) @ V, W)
            L_[0, j] = (next_f_j - curr_f) / epsilon
    elif arg.lower() == 'w' or arg == 1:
        curr_f = f(V, W)
        for j in range(6):
            zeta = I[:, j]
            next_f_j = f(V, expm(S(zeta) * epsilon) @ W)
            L_[0, j] = (next_f_j - curr_f) / epsilon
    else:
        raise ValueError("Invalid argument 'arg'. Use 'V' (0) or 'W' (1).")
    return L_

def XI(curve_parametrization, i, ds=0.01):
    """XI operator (See Definition 2.5). This function returns the necessary 
    twist in order to move from the i-th point to the (i+1)-th point in the 
    curve. This is an approximation of the 'derivative' of the curve. Parameter
    'i' will be used as the index of the nearest point in the curve throughout
    the script.

    Parameters
    ----------
    curve_parametrization : np.array
        Array with the precomputed curve. The shape is (n_points, 4, 4).
    i : int
        Index of the point in the curve.
    ds : float, optional
        Small value to compute the variation. The default is 0.01.
    
    Returns
    -------
    XI_ : np.array
        The corresponding twist vector xi.
    """
    # get G
    G = curve_parametrization[i]
    # If the index is the last, then the next point is the first in the curve.
    # Otherwise, the next point is the next in the curve
    if i == len(curve_parametrization) - 1:
        G_next = curve_parametrization[0]
    else:
        G_next = curve_parametrization[i + 1]
    # Compute the 'derivative'
    dGdsigma = (G_next - G) / ds
    XI_ = S_inv(dGdsigma @ np.linalg.inv(G))
    return XI_

def k_N(distance, k1=1.0, k2=20.0):
    """The gain of the normal component (k_N). This is a function of the minimum
    distance between the system and the curve. This is the same function as in
    section 4.4 of the paper.

    Parameters
    ----------
    distance : float
        Minimum distance between the system and the curve.
    k1 : float, optional
        Internal gain of the function. The default is 1.
    k2 : float, optional
        Internal gain of the function. The default is 20.
    
    Returns
    -------
    float
        The gain of the normal component.
    """
    return k1 * np.tanh(k2 * np.sqrt(distance))

def k_T(distance, k1=0.5, k2=1.0, k3=20.0):
    """The gain of the tangent component (k_T). This is a function of the minimum
    distance between the system and the curve. This is the same function as in
    section 4.4 of the paper.

    Parameters
    ----------
    distance : float
        Minimum distance between the system and the curve.
    k1 : float, optional
        Internal gain of the function. The default is 0.5.
    k2 : float, optional
        Internal gain of the function. The default is 1.
    k3 : float, optional
        Internal gain of the function. The default is 20.
    
    Returns
    -------
    float
        The gain of the tangent component.
    """
    return k1 * (1 - k2 * np.tanh(k3 * np.sqrt(distance)))

def gain_factory(gain, *args, **kwargs):
    """Function factory to return the gains with only one argument (distance).
    This is used to simplify the simulation function.

    Parameters
    ----------
    gain : function
        The gain function.
    *args : list
        Arguments of the gain function.
    **kwargs : dict
        Keyword arguments of the gain function.
    
    Returns
    -------
    gain_func : function
        The gain function with only one argument (distance).
    """
    def gain_func(distance):
        return gain(distance, *args, **kwargs)
    return gain_func

R = 0.4
r = 0.1
N_points = 1000
n_frames = 10 # aesthetics only

sphere_center = np.array([-2*R, 0., R])
circle_center = np.array([R, 0, 0.])
htm_sphere = Utils.trn(sphere_center)

# Rotate a reference vector around a given axis, such that the drawn circle
# has radius r
normal = (circle_center - sphere_center) / np.linalg.norm(circle_center - sphere_center)
ref_point = np.array([0, 0, R]).reshape(-1, 1)
ref_point = R * ref_point / np.linalg.norm(ref_point)
phi = np.arccos(r / R)
ref_H = Utils.roty(phi) @ np.array(Utils.trn(ref_point))
theta = np.linspace(0, 2*np.pi, N_points)
rotations = [R_theta(t, normal) for t in theta]
circle_htms = np.array([np.array(Utils.trn(sphere_center) @ Utils.roty(np.deg2rad(-45)) @ Q @ ref_H @ Utils.roty(np.pi)) for Q in rotations])

points = [np.array(H[:3, 3]).ravel() for H in circle_htms]
frames = []
idxs = np.linspace(0, len(circle_htms)-1, 10).astype(int)

for H in np.array(circle_htms)[idxs]:
    frames.append(Frame(htm=H, size=0.1))

circle = PointCloud(points=points, color='cyan', size=0.03)
sphere = Ball(htm=htm_sphere, radius=R, color='magenta', opacity=0.3)

kuka = ub.robot.Robot.create_kuka_kr5()
q_max = kuka.joint_limit[:, 1]
q_min = kuka.joint_limit[:, 0]
sim = ub.Simulation.create_sim_grid([circle, sphere, kuka])
sim.add(frames)

### Control loop
kn2 = 10.0
k_N = gain_factory(k_N, k1=1.0, k2=kn2)
k_T = gain_factory(k_T, k1=0.2, k2=1.0, k3=kn2)
H0 = kuka.fkm().copy()
curve = np.array(circle_htms)
T = 15
dt = 0.01
ds = 1 / len(curve)

imax = int(T/dt)
H_hist = []
distance_hist = []
nearest_point_hist = []
vector_field_hist = []
violations = []
q_hist = []

for i in range(imax):
    progress_bar(i, imax)
    J, H = kuka.jac_geo()
    J = np.array(J).copy()
    H = np.array(H).copy()
    q = np.array(kuka.q.copy())
    violated = (np.nonzero((q > q_max) | (q < q_min))[0]).any()
    violations.append(violated)
    # Compute the minimum distance and the nearest index:
    min_distance, min_index = D(H, curve)
    # Compute the nearest point:
    Hd_star = curve[min_index]
    # Compute the normal component:
    xi_N = -L(D_hat, H, Hd_star, 'V').T
    # Compute the tangent component:
    xi_T = XI(curve, min_index, ds=ds)
    # Compute the vector field:
    psi = k_N(min_distance) * xi_N + k_T(min_distance) * xi_T
    twist_ = psi.copy()
    v = twist_[:3] - np.array(Utils.S(twist_[3:]) @ (H[:3, 3]).reshape(-1, 1)).reshape(-1, 1)
    twist_[:3] = v
    # Compute qdot
    q_dot = np.array(Utils.dp_inv(J) @ twist_)
    kuka.add_ani_frame(time=i * dt, q=q + q_dot * dt)
    H_hist.append(H)
    distance_hist.append(min_distance)
    nearest_point_hist.append(Hd_star)
    vector_field_hist.append(psi)
    q_hist.append(q)

sim.run()

## PLOT CONFIGURATION AND LIMITS

import plotly.graph_objects as go

colorscale = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
q_hist_ = np.array(q_hist).reshape(-1, 6)
fig = go.Figure()
time_ = np.arange(len(q_hist_)) * dt

fig.add_trace(go.Scatter(x=time_,
                         y=np.array(violations) * np.max(q_hist_),
                         name='violations', line=dict(width=2, color='red'),
                         fill='tozeroy', fillcolor='rgba(255, 0, 0, 0.1)'),)

for i, qi in enumerate(q_hist_.T):
    fig.add_trace(go.Scatter(x=time_, y=qi, name=f'q{i+1}', mode='lines',
                             line=dict(width=2, color=colorscale[i])),)
    fig.add_trace(go.Scatter(x=time_, y=[q_max[i].item()] * len(time_), name=f'q{i+1}max', mode='lines',
                             line=dict(width=2, color=colorscale[i], dash='dash')),)
    fig.add_trace(go.Scatter(x=time_, y=[q_min[i].item()] * len(time_), name=f'q{i+1}min', mode='lines', 
                             line=dict(width=2, color=colorscale[i], dash='dot'),),)

fig.update_xaxes(title_text="Time (s)", gridcolor='gray', zerolinecolor='gray')
fig.update_yaxes(title_text="Joint position (rad)", gridcolor='gray',
                 zerolinecolor='gray', title_standoff=30)
fig.update_layout(margin=dict(l=20, r=0, t=20, b=0),)


fig.show()

### PLOT TASK FUNCTION

import plotly.graph_objects as go
from plotly.subplots import make_subplots

closest_points = nearest_point_hist
states = H_hist
distances = distance_hist

ori_errs = []
pos_errs = []
# Compute the distance, position error, and orientation error
for closest_point, state in zip(closest_points, states):
    p_near = closest_point[:3, 3]
    ori_near = closest_point[:3, :3]
    p_curr = state[:3, 3]
    ori_curr = state[:3, :3]
    pos_errs.append(np.linalg.norm(p_near - p_curr) * 100)
    trace_ = np.trace(ori_near @ np.linalg.inv(ori_curr))
    acos = np.arccos((trace_ - 1) / 2)
    # checks if acos is nan
    if np.isnan(acos):
        acos = 0
    ori_errs.append(acos * 180 / np.pi)

# Create a figure with three plots, one above another. First the distance, 
# then position error, and the orientation error
time_vec = np.arange(0, len(pos_errs) * dt, dt)
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x=time_vec, y=distances, showlegend=False, line=dict(width=3)), row=1, col=1)
fig.add_trace(go.Scatter(x=time_vec, y=pos_errs, showlegend=False, line=dict(width=3)), row=2, col=1)
fig.add_trace(go.Scatter(x=time_vec, y=ori_errs, showlegend=False, line=dict(width=3)), row=3, col=1)
fig.update_xaxes(title_text="Time (s)", gridcolor='gray', zerolinecolor='gray', row=3, col=1)
fig.update_xaxes(title_text="", gridcolor='gray', zerolinecolor='gray', row=1, col=1)
fig.update_xaxes(title_text="", gridcolor='gray', zerolinecolor='gray', row=2, col=1)
fig.update_yaxes(title_text="Distance D", gridcolor='gray', zerolinecolor='gray', 
                 row=1, col=1, title_standoff=30)
fig.update_yaxes(title_text="Pos. error (cm)", gridcolor='gray', zerolinecolor='gray', 
                 row=2, col=1, title_standoff=30)
fig.update_yaxes(title_text="Ori. error (deg)", gridcolor='gray', zerolinecolor='gray', 
                 row=3, col=1, title_standoff=30)

fig.update_layout(width=718.110, height=605.9155)
fig.show()

# Problem 3

In [ ]:
import uaibot as ub
import numpy as np
from uaibot.simobjects import Ball, Frame
from uaibot.utils import Utils


def sphere_collision_models(robot, q):
    """Computes collision models using spheres for Staubli robot at
    configuration 'q'

    Parameters:
    -----------
    robot: uaibot.Robot
        Staubli object
    q: np.npdarray
        The configuration

    Returns:
    --------
    col_models: list
        A list with every collision graph_objects
    """
    q = np.array(q)
    htms = robot.fkm(axis="dh", q=q)
    col_models = []

    # Link 1
    col_model_1 = [
        Ball(htm=htms[0], radius=0.1, opacity=0.5),
        Ball(htm=htms[0] @ Utils.trn([0, -0.07, 0]), radius=0.1, opacity=0.5),
        Ball(htm=htms[0] @ Utils.trn([0, -0.13, 0]), radius=0.1, opacity=0.5),
        Ball(htm=htms[0] @ Utils.trn([0, 0, -0.07]), radius=0.1, opacity=0.5),
        Ball(htm=htms[0] @ Utils.trn([0, 0, -0.13]), radius=0.1, opacity=0.5),
    ]

    # Link 2
    col_model_2 = [
        Ball(
            htm=htms[1] @ Utils.trn([0, 0, -0.07]),
            radius=0.1,
            opacity=0.5,
            color="blue",
        ),
        Ball(
            htm=htms[1] @ Utils.trn([-0.1, 0, -0.12]),
            radius=0.08,
            opacity=0.5,
            color="blue",
        ),
        Ball(
            htm=htms[1] @ Utils.trn([-0.2, 0.04, -0.12]),
            radius=0.08,
            opacity=0.5,
            color="blue",
        ),
        Ball(
            htm=htms[1] @ Utils.trn([-0.2, -0.04, -0.12]),
            radius=0.08,
            opacity=0.5,
            color="blue",
        ),
    ]

    # Link 3
    col_model_3 = [
        Ball(
            htm=htms[2] @ Utils.trn([0, 0, 0.03]),
            radius=0.11,
            opacity=0.5,
            color="green",
        ),
        Ball(
            htm=htms[2] @ Utils.trn([0, -0.03, -0.06]),
            radius=0.11,
            opacity=0.5,
            color="green",
        ),
    ]

    # Link 4
    col_model_4 = [
        Ball(
            htm=htms[3] @ Utils.trn([0, -0.02, 0]),
            radius=0.08,
            opacity=0.5,
            color="magenta",
        ),
        Ball(
            htm=htms[3] @ Utils.trn([0, -0.1, 0]),
            radius=0.08,
            opacity=0.5,
            color="magenta",
        ),
        Ball(
            htm=htms[3] @ Utils.trn([0, -0.15, 0]),
            radius=0.08,
            opacity=0.5,
            color="magenta",
        ),
    ]

    # Link 5
    col_model_5 = [
        Ball(
            htm=htms[4] @ Utils.trn([0, 0.0, 0.04]),
            radius=0.04,
            opacity=0.5,
            color="cyan",
        ),
    ]

    col_models.extend(col_model_1)
    col_models.extend(col_model_2)
    col_models.extend(col_model_3)
    col_models.extend(col_model_4)
    col_models.extend(col_model_5)

    return col_models


rng = np.random.default_rng(42)
q = rng.random(size=(6, 1)) * np.pi
print(q)
staubli = ub.Robot.create_staubli_tx60(opacity=0.7)
# q = staubli.q.copy()
staubli.set_ani_frame(q=q)
staubli.update_col_object(time=0)

sim = ub.simulation.Simulation.create_sim_grid([staubli])
col_models = sphere_collision_models(staubli, q)
# for link in staubli.links:
#     for colobj in link.col_objects:
#         sim.add(colobj[0])
sim.add(col_models)
sim.run()

# Problem 4

Tetrahedron depicted in gray, Octahedron in green, Dodecahedron in red and Icosahedron in cyan

In [ ]:
import uaibot as ub
import numpy as np
from uaibot.simobjects import Ball, Frame
from uaibot.graphics import MeshMaterial
from uaibot.utils import Utils
from itertools import combinations

def create_glass(color="#c0e0f0"):
    glass_material = MeshMaterial(
        # Core transparency properties
        opacity=1,                  # Slight opacity for "frosted glass" effect
        transmission=0.95,            # High light transmission (key for glass)
        roughness=0.05,               # Smooth surface (near-mirror finish)
        metalness=0,                  # Non-metallic
        
        # Refraction/reflection
        ior=1.52,                     # Index of refraction for glass (1.5-1.6)
        reflectivity=0.5,             # Moderate base reflectivity
        env_map_intensity=1.5,        # Boost environment reflections
        refraction_ratio=0.98,        # Slight refraction distortion
        
        # Surface details
        clearcoat=0.1,                # Thin reflective layer
        clearcoat_roughness=0.1,      # Slight imperfections
        specular_intensity=0.3,       # Highlight sharpness
        
        # Color/lighting
        color=color,              # Pale blue tint (hex or "white")
        emissive_intensity=0.5,         # No self-illumination
        flat_shading=False,           # Smooth normals
        
        # Advanced (optional)
        normal_scale=[0.5, 0.5],      # Subtle surface variation
        side="DoubleSide"             # Render both faces
    )
    return glass_material

# Tetraedron
## Vertices
L = 1
h = L / np.sqrt(2)
a = L / np.sqrt(3)
v = np.array([
    [0, 0, h],
    [a, 0, 0],
    [-a/2, a/2 * np.sqrt(3), 0],
    [-a/2, -a/2 * np.sqrt(3), 0]
])

interior_point = np.mean(v, axis=0)
## Normal vectors
normals = []
n_faces = 4
idxs = list(combinations(range(v.shape[0]), n_faces - 1))
A = []

for idx in idxs:
    p1 = v[idx[0]]
    p2 = v[idx[1]]
    p3 = v[idx[2]]
    v1 = p2 - p1
    v2 = p3 - p1
    n = np.cross(v1, v2)
    n = n / np.linalg.norm(n)
    if np.dot(n, interior_point) > 0:
        n = -n
    A.append(n.ravel())

A = np.array(A)
b = np.ones((A.shape[0], 1)) * (-0.5)
b[-1] = 0

tetraedron = ub.ConvexPolytope(A=A, b=b, mesh_material=create_glass())
tetraedron.set_ani_frame(htm=Utils.trn([0, -2, 1]))

# Octahedron
angle = np.pi/2
normal = ub.Utils.rotx(angle / 2) @ np.array([0, 1, 0, 1]).reshape(-1, 1)
normal = np.array(normal)
sim = ub.Simulation.create_sim_grid([tetraedron])
A = []
for i in range(4):
    n_up_ = ub.Utils.rotz(np.pi/2 * i) @ ub.Utils.rotx(angle / 2) @ Utils.trn([0, 1, 0])
    A.append(np.array(n_up_ @ np.array([0,1,0,1]).reshape(-1, 1)).ravel()[:3])

    n_down_ = ub.Utils.rotz(np.pi/2 * i) @ ub.Utils.rotx(-angle / 2) @ Utils.trn([0, 1, 0])
    A.append(np.array(n_down_ @ np.array([0,1,0,1]).reshape(-1, 1)).ravel()[:3])

A = np.array(A)
b = np.ones((len(A), 1)) * (1)

octaedron = ub.ConvexPolytope(A=A, b=b, mesh_material=create_glass(color='lime'))
octaedron.set_ani_frame(htm=Utils.trn([2, 0, 1]))
sim.add(octaedron)

# Dodecahedron
## Vertices
phi = (1 + np.sqrt(5)) / 2
v = np.array([
    [1, 1, 1], # A0
    [1, 1, -1], # A1
    [1, -1, 1], # A2
    [1, -1, -1], # A3
    [-1, 1, 1], # A4
    [-1, 1, -1], # A5
    [-1, -1, 1], # A6
    [-1, -1, -1], # A7
    [0, phi, 1/phi], # A8
    [0, phi, -1/phi], # A9
    [0, -phi, 1/phi], # A10
    [0, -phi, -1/phi], # A11
    [phi, 1/phi, 0], # A12
    [phi, -1/phi, 0], # A13
    [-phi, 1/phi, 0], # A14
    [-phi, -1/phi, 0], # A15
    [1/phi, 0, phi], # A16
    [-1/phi, 0, phi], # A17
    [1/phi, 0, -phi], # A18
    [-1/phi, 0, -phi] # A19
])

interior_point = np.mean(v, axis=0)
## Normal vectors
# list of tuples of vertices idexes on the same face

faces = [
    (17, 0, 4, 8, 16),
    (17, 2, 6, 10, 16),
    (17, 4, 6, 14, 15),
    (0, 1, 8, 9, 12),
    (0, 2, 12, 13, 16),
    (1, 3, 12, 13, 18),
    (1, 5, 9, 18, 19),
    (2, 3, 10, 11, 13),
    (3, 7, 11, 18, 19),
    (4, 5, 8, 9, 14),
    (5, 7, 14, 15, 19),
    (6, 7, 10, 11, 15),

]

A = []
for idx in faces:
    p1 = v[idx[0]]
    p2 = v[idx[1]]
    p3 = v[idx[2]]
    v1 = p2 - p1
    v2 = p3 - p1
    n = np.cross(v1, v2)
    n = n / np.linalg.norm(n)
    if np.dot(n, interior_point - p1) > 0:
        n = -n
    A.append(n.ravel())

A = np.array(A)
b = np.ones((A.shape[0], 1)) * (0.5)

dodecahedron = ub.ConvexPolytope(A=A, b=b, mesh_material=create_glass(color='red'))
dodecahedron.set_ani_frame(htm=Utils.trn([0, 2, 1]))
sim.add(dodecahedron)

# Icosahdron
v = np.array([
    [0, 1, phi], 
    [0, -1, phi], 
    [0, 1, -phi], 
    [0, -1, -phi],
    [1, phi, 0], 
    [-1, phi, 0],
    [1, -phi, 0],
    [-1, -phi, 0],
    [phi, 0, 1], 
    [phi, 0, -1],
    [-phi, 0, 1],
    [-phi, 0, -1]
])

faces = [
    # Upper pentagon cap (5 faces)
    (0, 1, 8), (0, 8, 4), (0, 4, 5), (0, 5, 10), (0, 10, 1),
    
    # Middle band (10 faces)
    (1, 8, 6), (8, 4, 9), (4, 5, 2), (5, 10, 11), (10, 1, 7),
    (1, 6, 7), (8, 6, 9), (4, 9, 2), (5, 2, 11), (10, 11, 7),
    
    # Lower pentagon cap (5 faces)
    (6, 9, 3), (9, 2, 3), (2, 11, 3), (11, 7, 3), (7, 6, 3)
]

A = []
for idx in faces:
    p1 = v[idx[0]]
    p2 = v[idx[1]]
    p3 = v[idx[2]]
    v1 = p2 - p1
    v2 = p3 - p1
    n = np.cross(v1, v2)
    n = n / np.linalg.norm(n)
    if np.dot(n, interior_point - p1) > 0:
        n = -n
    A.append(n.ravel())
# A.append([0, 0, -1])
A = np.array(A)
b = np.ones((A.shape[0], 1)) * (0.5)

icosahedron = ub.ConvexPolytope(A=A, b=b, mesh_material=create_glass(color='cyan'))
icosahedron.set_ani_frame(htm=Utils.trn([-2, 0, 1]))
sim.add(icosahedron)

sim.run()

# Problem 5

LiDAR is depicted as a "glassy" cone, and detected points are shown in cyan

In [ ]:
import uaibot as ub
import numpy as np
from uaibot.graphics import MeshMaterial
from uaibot.simobjects import Ball, Box, Frame, Cylinder, PointCloud

def generate_cone_constraints(height, theta, num_faces=8):
    """Generate polytope constraints (A, b) for a cone with given height and
    opening angle.
    
    Parameters:
    ----------
    height: float 
        Distance from apex to base along z-axis
    theta: float
        The opening angle (rad)
    num_faces: int
        Number of faces to approximate the cone (>= 3)
    
    Returns:
    --------
        A: np.array
            (num_faces+1, 3) constraint matrix
        b: np.array
            (num_faces+1,) constraint vector
    """
    half_angle = theta / 2.0
    radius = height * np.tan(half_angle)
    
    # Generate circular base constraints
    A = []
    b = []
    
    for i in range(num_faces):
        theta = 2 * np.pi * i / num_faces
        # Normal vector for side face
        nx = np.cos(theta)
        ny = np.sin(theta)
        nz = radius/height  # = tan(pi/2 - half_angle)
        
        # Normalize the normal vector
        norm = np.sqrt(nx**2 + ny**2 + nz**2)
        A.append([nx/norm, ny/norm, nz/norm])
        b.append(0.0)  # All faces intersect at origin
    
    # Add base constraint (only if height is finite)
    if np.isfinite(height):
        A.append([0, 0, -1])  # Points must have z ≤ height
        b.append(height)
    
    return np.array(A), np.array(b)

def create_glass(color="#c0e0f0"):
    glass_material = MeshMaterial(
        # Core transparency properties
        opacity=1,                  # Slight opacity for "frosted glass" effect
        transmission=0.95,            # High light transmission (key for glass)
        roughness=1,               # Smooth surface (near-mirror finish)
        metalness=0,                  # Non-metallic
        
        # Refraction/reflection
        ior=1.52,                     # Index of refraction for glass (1.5-1.6)
        reflectivity=0.,             # Moderate base reflectivity
        env_map_intensity=0,        # Boost environment reflections
        refraction_ratio=0.98,        # Slight refraction distortion
        
        # Surface details
        clearcoat=0.0,                # Thin reflective layer
        clearcoat_roughness=0.1,      # Slight imperfections
        specular_intensity=0.3,       # Highlight sharpness
        
        # Color/lighting
        color=color,              # Pale blue tint (hex or "white")
        emissive_intensity=0.5,         # No self-illumination
        flat_shading=False,           # Smooth normals
        
        # Advanced (optional)
        normal_scale=[0.5, 0.5],      # Subtle surface variation
        side="DoubleSide"             # Render both faces
    )
    return glass_material

#Create the object
L = 0.1
fov = np.deg2rad(40)
obj = Ball(htm = ub.Utils.trn([1, -0.5, 0.5]), radius=0.5)
#Sample using discretization disc=0.25
obj_pc = obj.to_point_cloud(disc=0.01)

box_htm = ub.Utils.trn([0,-0.5, L/2]) @ ub.Utils.roty(np.deg2rad(60))
lidar = Box(htm = box_htm, height=L, width=L, depth=L, color='red', opacity=1)
lidar_htm = np.array(box_htm @ ub.Utils.trn([0, 0, L/2]))
lidar_frame = Frame(htm=lidar_htm, size=0.1)

p = np.array(lidar_htm[:3, 3])
pz = lidar_htm[:3, 2] #(lidar_htm @ np.array([0, 0, 1, 1]).reshape(-1, 1))[:3]

detections = []
for i, pc_i in enumerate(obj_pc.points.T):
    pc_v = (np.array(pc_i).ravel() - p.ravel()).reshape(-1, 1)
    angle = np.arccos((pc_v.T @ pz) / (np.linalg.norm(pc_v) * np.linalg.norm(pz)))
    if abs(angle) <= fov/2:
        # Check occlusion
        alpha = 0.95
        end_point = alpha * pc_i + (1 - alpha) * p
        middle_point = (p + pc_i) / 2
        ray_height = np.linalg.norm(end_point - p)
        z_ax = ((end_point - p) / ray_height).reshape(-1, 1)
        x_ax = (ub.Utils.rotz(np.pi/2) @ np.hstack((z_ax.ravel(), 1)).reshape(-1, 1))[:3]
        y_ax = np.cross(z_ax.ravel(), x_ax.ravel()).reshape(-1, 1)
        ray_htm = ub.Utils.trn(middle_point)
        ray_htm[:3, :3] = np.hstack((x_ax, y_ax, z_ax))

        length = np.linalg.norm(p.ravel() - end_point.ravel())
        ray = Cylinder(htm=ray_htm, radius=obj_pc.size/2, height=length, color='cyan')
        frame = Frame(htm=ray_htm, size=0.3)
        *_, dist, _ = ray.compute_dist(obj)
        if dist > 1e-5:
            detections.append(i)

detected_points = PointCloud(points=obj_pc.points[:, detections], color='cyan', 
                             size=obj_pc.size)
A, b = generate_cone_constraints(height=4, theta=fov, num_faces=8,)
cone = ub.ConvexPolytope(A=A, b=b, opacity=0.3,
                         mesh_material=create_glass(color="#c0d8ff"))
cone.set_ani_frame(htm=ub.Utils.trn([lidar_htm[0, 3], 0, box_htm[2, 3] 
                       + lidar_htm[2, 3]]) @ ub.Utils.roty(np.pi) @ box_htm @ (
                        ub.Utils.trn([0, 0, 4]) @ cone.htm 
                        @ ub.Utils.trn([-4, 0, 0])))
# Remove detected points from the point cloud to reduce rendering
# and improve performance
obj_pc._points = np.delete(obj_pc.points, detections, axis=1)

sim = ub.Simulation.create_sim_grid([obj_pc, lidar, lidar_frame, detected_points, cone])
sim.set_parameters(camera_start_pose = [-1, -0.6 ,0.6, 5, 0, 0, 1])
# sim.add([ray1, ray2, ray3, ray4])
sim.run()

# Problem 6

Steps were animated using Plotly (the flickering seems inevitable)

In [ ]:
import numpy as np
import plotly.graph_objects as go

rng = np.random.default_rng(42)
H = rng.normal(size=(2, 2)) * 2
H = H @ H.T

f = rng.random(size=(2, 1)) * 2

# Unit square polytope, centered at the origin
A = np.array([
    [1, 0],
    [0, 1],
    [-1, 0],
    [0, -1],
])
b = np.array([-0.5, -0.5, -0.5, -0.5]).reshape(-1, 1)

def obj_fun(u, H, f):
    u = np.array(u).reshape(-1, 1)
    H = np.array(H)
    f = np.array(f).reshape(-1, 1)
    return (0.5 * u.T @ H @ u + f.T @ u).item()

# Projected gradient descent
def project_gradient_descent(H, f, A, b, alpha=0.1, max_iter=1000, tol=1e-5, hist_x=[], hist_fun=[]):
    H = np.array(H)
    f = np.array(f).reshape(-1, 1)
    A = np.array(A)
    b = np.array(b).reshape(-1, 1)

    x = np.zeros((H.shape[0], 1))
    fun = obj_fun(x, H, f)

    for i in range(max_iter):
        grad = (0.5 * H @ x) + f
        x = x - alpha * grad

        for j in range(A.shape[0]):
            aj = A[j].reshape(1, -1)
            if aj @ x < b[j]:
                x = x - (aj @ x - b[j]) * aj.reshape(-1, 1) / (aj @ aj.T)
        
        hist_x.append(x.copy())
        fun_ = obj_fun(x, H, f)
        hist_fun.append(fun_)
        if abs(fun - fun_) < tol:
            break
        fun = fun_ 

    return x, hist_x, hist_fun

x, hist_x, hist_fun = project_gradient_descent(H, f, A, b, max_iter=50000, 
                                               alpha=8e-2, tol=1e-5)
print(f'optimal sol u^*: {x.ravel()}')
hist_x = np.array(hist_x).reshape(-1, 2)

# Plotting
xs = np.linspace(-2, 2, 20)
X, Y = np.meshgrid(xs, xs)
Z = np.zeros_like(X)

# Evaluate obj_fun at each (x,y) point
for i in range(len(x)):
    for j in range(len(xs)):
        u = np.array([X[i,j], Y[i,j]])
        Z[i,j] = obj_fun(u, H, f)


Z = np.array([[obj_fun(np.array([x, y]), H, f) for x in xs] for y in xs])
contour = go.Contour(z=Z, x=xs, y=xs, ncontours=20, colorscale='rdbu',)
contour_aux = go.Contour(z=Z, x=xs, y=xs, ncontours=20, colorscale='RdBu')
fig = go.Figure(data=[contour])

fig.add_scatter(x=[-0.5, -0.5, 0.5, 0.5, -0.5], y=[-0.5, 0.5, 0.5, -0.5, -0.5], 
                mode='lines', fill='toself', name='polytope', line=dict(
                    width=1.3, color='gray'), hoverinfo='skip', showlegend=False)

fig.add_trace(go.Scatter(
            x=[],
            y=[],
            mode='lines+markers',
            marker=dict(color='black', size=8),
            line=dict(color='black', width=2),
            name='Gradient Descent'
        ))

frames = []
for i, point in enumerate(hist_x):
    scatter = go.Scatter(x=hist_x[:i, 0], y=hist_x[:i, 1], mode='lines+markers', 
                         marker=dict(color='black', size=8), 
                         line=dict(color='black', width=2),
                         name='Gradient Descent', showlegend=False,
                         )
    frames.append(go.Frame(data=[scatter], traces=[2], name=f'frame_{i}'))


fig.update_layout(
    updatemenus=[{
        'type': 'buttons',
        'buttons': [
            {
                'label': '▶️ Play',
                'method': 'animate',
                'args': [None, dict(
                    # mode='immediate',
                    frame=dict(duration=300, redraw=True),
                    fromcurrent=True,
                    transition=dict(duration=0, easing="cubic-in-out")
                )]
            },
            {
                'label': '⏹️ Pause',
                'method': 'animate',
                'args': [[None], dict(
                    mode='immediate',
                    frame=dict(duration=0, redraw=False),
                    transition=dict(duration=0)
                )]
            }
        ]
    }],
    sliders=[{
        'active': 0,
        'steps': [
            {
                'args': [
                    [f.name],
                    {
                        'frame': {'duration': 0, 'redraw': True},
                        'mode': 'immediate'
                    }
                ],
                'label': f'Step {i}',
                'method': 'animate'
            } for i, f in enumerate(frames)
        ],
        'transition': {'duration': 0},
    }]
)
fig.frames = frames
fig.show()

# Problem 7

Algorithm failed in 9 cases during the last run.

In [ ]:
import numpy as np
import uaibot as ub
import plotly.graph_objects as go
from uaibot.simobjects import Ball, Box, Cylinder

def create_random_object(rng):
    obj_type = rng.choice(["Cylinder", "box", "ball"])
    htm = ub.Utils.htm_rand(trn_min=[-5., -5., 0], trn_max = [5., 5., 4], rot=np.pi)
    if obj_type == "Cylinder":
        radius = rng.uniform(low=0.2) * 3
        height = rng.uniform(low=0.2) * 4
        obj = Cylinder(htm=htm, radius=radius, height=height)
    elif obj_type == "box":
        width = rng.uniform(low=0.2) * 4
        depth = rng.uniform(low=0.2) * 4
        height = rng.uniform(low=0.2) * 4
        obj = Box(htm=htm, width=width, depth=depth, height=height)
    else:
        radius = rng.uniform(low=0.2) * 3
        obj = Ball(htm=htm, radius=radius)
    
    return obj
    
max_iter = 4000
works = []

for trial in range(10000):
    rng = np.random.default_rng(42 + trial)
    obj1 = create_random_object(rng)
    obj2 = create_random_object(rng)

    point = np.matrix([[1],[2],[3]])
    old_point = np.matrix([[0],[0],[0]])
    dist = np.inf    
    i = 0

    while np.linalg.norm(point - old_point) > 1e-6 and i < max_iter:
        proj, dist = obj1.projection(point)
        proj, dist = obj2.projection(proj)
        old_point, point = point, proj
        i += 1

    works.append(i)


fig = go.Figure(data=[go.Histogram(x=works, histfunc="count", nbinsx=25)])
fig.update_xaxes(title_text="Steps")
fig.update_yaxes(title_text="Count")
fig.show()

print(f"Mean: {np.mean(works):.2f} ± {np.std(works):.2f} steps")
print(f"Number of steps greater than {max_iter} in {np.sum(np.array(works) >= max_iter)} cases")

# Problem 8

Along with the average number of leaves evaluated and percentage compared to brute-force, we also plot the last circle and point cloud considered, and the resulting distance.

In [ ]:
import numpy as np
import plotly.graph_objects as go
from collections import deque

def random_point_cloud(n_points, dim=2, mean=1, std=1, seed=None):
    """Generates a random dim-dimensional point cloud with n_points points.
    The points are generated from a normal distribution with mean and std.

    Parameters:
    -----------
    n_points: int
        Number of points in the point cloud
    dim: int
        Dimension of the point cloud
    mean: float
        Mean of the normal distribution
    std: float
        Standard deviation of the normal distribution
    
    Returns:
    --------
    points: np.ndarray
        A (n_points, dim) array with the points in the point cloud
    """

    rng = np.random.default_rng(seed)
    points = rng.normal(loc=mean, scale=std, size=(n_points, dim))
    return points


class Circle:
    def __init__(self, n_points, max_radius=1, center_bounds=(0, 0), seed=None):
        self.points, self.radius, self.center = self.random_circle(n_points, max_radius, center_bounds, seed)
        self.aabb = create_aabb(self.points)

    def random_circle(self, n_points, max_radius=1, center_bounds=(0, 0), seed=None):
        """Generates a random circle with n_points points with a random radius
        and center.
        """
        
        rng = np.random.default_rng(seed)
        angles = np.linspace(0, 2 * np.pi, n_points)
        radius = rng.uniform(low=0, high=max_radius)
        center = rng.uniform(low=center_bounds[0], high=center_bounds[1], size=(2,))
        x = center[0] + radius * np.cos(angles)
        y = center[1] + radius * np.sin(angles)
        
        points = np.column_stack((x, y))
        return points, radius, center

def create_aabb(points):
    """Creates a 2D axis-aligned bounding box (AABB) for a set of points.
    
    Parameters:
    -----------
    points: np.ndarray
        A (n_points, 2) array with the points in the point cloud
    
    Returns:
    --------
    aabb: np.ndarray
        A (4, 2) array with the min and max coordinates of the AABB.
        [[min_x, min_y], [max_x, max_y]]
    """
    min_coords = np.min(points, axis=0)
    max_coords = np.max(points, axis=0)
    aabb = np.vstack((min_coords, max_coords))
    return aabb

def aabb_distance(aabb1, aabb2):
    """
    Computes the distance between two AABBs.
    
    Parameters:
    -----------
    aabb1 : np.ndarray
        [[min_x, min_y], [max_x, max_y]]
    aabb2 : np.ndarray
        [[min_x, min_y], [max_x, max_y]]
    
    Returns:
    --------
    distance: float
        Distance between the two AABBs (0 if overlapping).
    """
    # Unpack AABBs
    (a1_min_x, a1_min_y), (a1_max_x, a1_max_y) = aabb1
    (a2_min_x, a2_min_y), (a2_max_x, a2_max_y) = aabb2
    
    # Check for overlap
    if (a1_max_x >= a2_min_x and a2_max_x >= a1_min_x and
        a1_max_y >= a2_min_y and a2_max_y >= a1_min_y):
        return 0.0  # AABBs overlap or touch
    
    # Compute separation along x and y axes
    dx = max(0, a2_min_x - a1_max_x, a1_min_x - a2_max_x)
    dy = max(0, a2_min_y - a1_max_y, a1_min_y - a2_max_y)
    
    distance = np.sqrt(dx**2 + dy**2)
    return distance

class Node:
    def __init__(self, points):
        self.points = points
        self.parent = None
        self.left = None
        self.right = None
        self.aabb = create_aabb(points)
        self.d = None

class BVH:
    def __init__(self, point_cloud):
        self.root = Node(point_cloud)
        self.build(self.root, mode=False)

    def _split_points(self, points, axis):
        if len(points) <= 1:
            return points, None
        
        # Sort points along the axis
        sorted_indices = np.argsort(points[:, axis])
        sorted_points = points[sorted_indices].copy()
        
        # Split into left and right halves
        mid = len(sorted_points) // 2
        left_points = sorted_points[:mid]
        right_points = sorted_points[mid:]
        
        return left_points, right_points
    
    def build(self, node, mode=False):
        """Recursively builds the BVH tree by splitting nodes.
        The splitting is done by alternating between horizontal (mode=False) and
        vertical (mode=True) splits.
        """
        if len(node.points) <= 1:
            return
        
        # Alternate splitting axis (0: x, 1: y)
        axis = 0 if not mode else 1
        
        left_points, right_points = self._split_points(node.points, axis)
        
        if left_points is not None and len(left_points) > 0:
            node.left = Node(left_points)
            node.left.parent = node
            self.build(node.left, mode=not mode)
        
        if right_points is not None and len(right_points) > 0:
            node.right = Node(right_points)
            node.right.parent = node
            self.build(node.right, mode=not mode)


def dist_circ_bvh(bvh, circle):
    """Computes the distance between a circle and a BVH."""
    lstar = np.inf
    queue = deque([bvh.root])
    leaves_eval = 0
    
    while queue:
        node = queue.popleft()
        
        node.d = aabb_distance(node.aabb, circle.aabb)
        
        if node.d >= lstar:
            continue
        
        # Leaf node -> compute distance point to circle
        if node.left is None and node.right is None:
            dist = np.linalg.norm(node.points.ravel() - circle.center.ravel())
            dist = max(0, dist - circle.radius)
            if dist < lstar:
                lstar = dist
            leaves_eval += 1
        else:
            if node.left:
                queue.append(node.left)
            if node.right:
                queue.append(node.right)
    
    return lstar, leaves_eval

leaves = []
N = 500
n_points = 500

for i in range(N):
    point_cloud = random_point_cloud(n_points, dim=2, mean=0, std=1, seed=i)
    bounding_volume = BVH(point_cloud)
    circle = Circle(1000, max_radius=0.5, center_bounds=(-2, -2), seed=i)
    dist, leaves_eval = dist_circ_bvh(bounding_volume, circle)
    leaves.append(leaves_eval)
            

fig = go.Figure(go.Scatter(
    x=point_cloud[:, 0],
    y=point_cloud[:, 1],
    mode='markers',
    marker=dict(size=5, color='blue'),
    name='Point Cloud'
))
fig.add_trace(go.Scatter(
    x=circle.points[:, 0],
    y=circle.points[:, 1],
    mode='lines',
    line=dict(width=2, color='red'),
    name='Circle'
))
fig.show()

print(f"Last distance: {dist:.2f}")
print(f"Average number of leaves evaluated over {N} runs: {np.mean(leaves):.2f} ± {np.std(leaves):.2f}")
print(f"Percentage of computations compared to brute force ({n_points} points): {np.mean(leaves)/n_points:.2%}")

# Problem 9

In [ ]:
import uaibot as ub
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from uaibot.simobjects import Ball, Frame, Box
from uaibot.utils import Utils

def sphere_collision_models(robot, q, htms=False):
    """Computes collision models using spheres for Staubli robot at
    configuration 'q'

    Parameters:
    -----------
    robot: uaibot.Robot
        Staubli object
    q: np.npdarray
        The configuration
    htms: bool
        If False, the function does not compute the forward kinematics
        and uses the identity matrix instead. If True, the function
        computes the forward kinematics and uses the resulting HTMs.

    Returns:
    --------
    col_models: list
        A list with every collision graph_objects
    """
    q = np.array(q)
    if htms == False:
        htms = [np.eye(4) for _ in range(len(q))]
    else:
        htms = robot.fkm(axis="dh", q=q)
    col_models = []

    # Link 1
    col_model_1 = [
        Ball(htm=htms[0], radius=0.1, opacity=0.5),
        Ball(htm=htms[0] @ Utils.trn([0, -0.07, 0]), radius=0.1, opacity=0.5),
        Ball(htm=htms[0] @ Utils.trn([0, -0.13, 0]), radius=0.1, opacity=0.5),
        Ball(htm=htms[0] @ Utils.trn([0, 0, -0.07]), radius=0.1, opacity=0.5),
        Ball(htm=htms[0] @ Utils.trn([0, 0, -0.13]), radius=0.1, opacity=0.5),
    ]

    # Link 2
    col_model_2 = [
        Ball(
            htm=htms[1] @ Utils.trn([0, 0, -0.07]),
            radius=0.1,
            opacity=0.5,
            color="blue",
        ),
        Ball(
            htm=htms[1] @ Utils.trn([-0.1, 0, -0.12]),
            radius=0.08,
            opacity=0.5,
            color="blue",
        ),
        Ball(
            htm=htms[1] @ Utils.trn([-0.2, 0.04, -0.12]),
            radius=0.08,
            opacity=0.5,
            color="blue",
        ),
        Ball(
            htm=htms[1] @ Utils.trn([-0.2, -0.04, -0.12]),
            radius=0.08,
            opacity=0.5,
            color="blue",
        ),
    ]

    # Link 3
    col_model_3 = [
        Ball(
            htm=htms[2] @ Utils.trn([0, 0, 0.03]),
            radius=0.11,
            opacity=0.5,
            color="green",
        ),
        Ball(
            htm=htms[2] @ Utils.trn([0, -0.03, -0.06]),
            radius=0.11,
            opacity=0.5,
            color="green",
        ),
    ]

    # Link 4
    col_model_4 = [
        Ball(
            htm=htms[3] @ Utils.trn([0, -0.02, 0]),
            radius=0.08,
            opacity=0.5,
            color="magenta",
        ),
        Ball(
            htm=htms[3] @ Utils.trn([0, -0.1, 0]),
            radius=0.08,
            opacity=0.5,
            color="magenta",
        ),
        Ball(
            htm=htms[3] @ Utils.trn([0, -0.15, 0]),
            radius=0.08,
            opacity=0.5,
            color="magenta",
        ),
    ]

    # Link 5
    col_model_5 = [
        Ball(
            htm=htms[4] @ Utils.trn([0, 0.0, 0.04]),
            radius=0.04,
            opacity=0.5,
            color="cyan",
        ),
    ]

    col_models.append(col_model_1)
    col_models.append(col_model_2)
    col_models.append(col_model_3)
    col_models.append(col_model_4)
    col_models.append(col_model_5)

    return col_models

def compute_distance(models, obstacles, jac, htm):
    if not isinstance(models, (list, tuple, np.ndarray)):
        models = [models]
    if not isinstance(obstacles, (list, tuple, np.ndarray)):
        obstacles = [obstacles]

    min_dist = np.inf
    distances = []
    derivatives = []
    Jv = np.array(jac[:3, :])
    Jw = np.array(jac[3:, :])
    p_model = np.array(htm[:3, 3]).reshape(-1, 1)

    for model in models:
        htm_model = model.htm
        p_model = htm_model[:3, 3].reshape(-1, 1)
        for obstacle in obstacles:
            witness_model, witness_obstacle, dist, *_ = model.compute_dist(obstacle)
            witness_model = np.array(witness_model).reshape(-1, 1)
            witness_obstacle = np.array(witness_obstacle).reshape(-1, 1)
            dist = float(dist)
            distances.append(dist)
    
            wit_diff = witness_model - witness_obstacle
         
            cross_term = np.array(Utils.S(witness_model - p_model)) @ (wit_diff)
            cross_term = np.array(cross_term).reshape(-1, 1)
            dD_dq = (wit_diff.T @ Jv) + (cross_term.T @ Jw)

            if dist < min_dist:
                min_dist = dist
                dqmin = dD_dq
            
            # if dist == min_dist:
            #     dD_dq = dD_dq * 0

            derivatives.append(dD_dq)
    return min_dist, dqmin, distances, derivatives

def compute_dist_robot_obstacles(col_models, obstacles, robot):
    dht_jacs, dht_htms = robot.jac_geo(axis='dh')
    distances = []
    dist_vec = []
    jac_D = []
    derivatives = []
    for i, models in enumerate(col_models):
        jac = dht_jacs[i]
        htm = dht_htms[i]
        min_dist, dD_dq, dists, ders = compute_distance(models, obstacles, jac, htm)
        distances.append(min_dist)
        derivatives.append(dD_dq)
        dist_vec.append(dists)
        jac_D.append(ders)
        
    robot_obstacle_dist = np.min(distances)
    dist_derivative =  derivatives[np.argmin(distances)]
    return robot_obstacle_dist, dist_derivative, dist_vec, jac_D

def update_col_models(time, col_models, q, robot, htm0_models):
    htms = robot.fkm(axis="dh", q=q)

    for i, ith_link_models in enumerate(col_models):
        for j, model in enumerate(ith_link_models):
            new_htm = htms[i] @ htm0_models[i][j]
            model.add_ani_frame(time=time, htm=new_htm)

def phi(r):
    r_ = np.array(r).copy()

    for i in range(r_.shape[0]):
        r_[i] = np.sign(r_[i]) * np.sqrt(np.abs(r_[i]))
        
    return r_

rng = np.random.default_rng(69)
q = rng.random(size=(6, 1)) * np.pi
staubli = ub.Robot.create_staubli_tx60()
staubli.set_ani_frame(q=q)
staubli.update_col_object(time=0)

sim = ub.simulation.Simulation.create_sim_grid([staubli])
col_models = sphere_collision_models(staubli, q, htms=False)
htm0_models = [[model.htm for model in models] for models in col_models]
update_col_models(0, col_models, q, staubli, htm0_models)

for i, models in enumerate(col_models):
    for model in models:
        sim.add(model)

h_obs = 1
obstacle = Box(htm=Utils.trn([0.36, 0, 1]), width=0.2, depth=0.2, height=h_obs)
target = Utils.trn([0.4, 0.3, 0.7]) @ Utils.roty(np.pi/2) @ Utils.rotx(-np.pi/4) 
target_frame = Frame(htm=target, size=0.3)
sim.add([obstacle, target_frame])

dt = 1e-2
eta = 3e-1
d_safe = 5e-3
epsilon = 1e-2
Kp = 2.0

min_dists = []
min_derivatives = []
r = np.ones((3, 1)) * 1e4
i = 1

while np.linalg.norm(r) > 1e-2:
    r, Jr = staubli.task_function(target)
    r = np.array(r).reshape(-1, 1)
    Jr = np.array(Jr)
    min_dist, min_grad, distances, jacobian = compute_dist_robot_obstacles(col_models, obstacle, staubli)
    min_dists.append(min_dist)
    b = np.array([d_ for dists_ in distances for d_ in dists_])
    A = np.array([g_.ravel() for grads_ in jacobian for g_ in grads_])
    b = -eta * (b - d_safe)
    H = 2*(Jr.T @ Jr + epsilon * np.eye(6))
    f = Kp * Jr.T @ phi(r)
    qdot = Utils.solve_qp(H.astype(np.double), f.astype(np.double), A.astype(np.double), b.astype(np.double))
    dDdt = min_grad @ qdot
    min_derivatives.append(dDdt)
    q = q + dt * qdot
    staubli.add_ani_frame(time=i*dt, q=q)
    update_col_models(i*dt, col_models, q, staubli, htm0_models)
    i += 1

sim.run()

fig = make_subplots(rows=2, cols=1, subplot_titles=("Distance", "Derivative"))
fig.add_trace(go.Scatter(y=min_dists, showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(y=np.array(min_derivatives).ravel(), showlegend=False),
              row=2, col=1)
fig.update_layout(margin=dict(l=0, r=0, t=20, b=0))
fig.show()